# Congressional Speech Analysis

### 1. Data import

In [59]:
import pandas as pd
# its necessary to run --> pip3 install xlrd==1.2.0               
df_charisma_analysis=pd.read_excel("charisma_analisis_complete_df.xlsx", header=0)
df_congress_114_bills=pd.read_excel("congress_114_bills_df_new.xlsx",header=0)
print(f'''charisma_analisis_complete has {df_charisma_analysis.shape[0]} Rows, and {df_charisma_analysis.shape[1]} Columns''')
print(f'''congress_114_bills has {df_congress_114_bills.shape[0]} Rows, and {df_congress_114_bills.shape[1]} Columns''')

charisma_analisis_complete has 10938 Rows, and 175 Columns
congress_114_bills has 3886 Rows, and 25 Columns


---
### 2. Cleaning data

##### 2.1 Indices 

In [2]:
print(f'''For df_congress_114_bills the field sentence_id has {sum(df_congress_114_bills.sentence_id.value_counts()>1)} repeted values\n''')
print(f'''While for df_charisma_analysis the field paragraph_id has {sum(df_charisma_analysis.paragraph_id.value_counts()==2)} repeted index''')

For df_congress_114_bills the field sentence_id has 0 repeted values

While for df_charisma_analysis the field paragraph_id has 24 repeted index


##### 2.2 Repeted paragraph_id

In [98]:
print('The list of id_paragraph repeated is:')
print(df_charisma_analysis.paragraph_id.value_counts()[df_charisma_analysis.paragraph_id.value_counts()>1])

The list of repeated id_paragraph is:
7492    2
3015    2
38      2
9841    2
5902    2
4910    2
6420    2
6950    2
8505    2
2693    2
7640    2
6911    2
9637    2
6788    2
6391    2
2851    2
3010    2
7990    2
4490    2
5219    2
5415    2
4858    2
1713    2
6621    2
Name: paragraph_id, dtype: int64


> The differences between this rows are in the features values.

In [102]:
df_charisma_analysis.loc[df_charisma_analysis.paragraph_id==7492]

,paragraph_id,paragraph,unique_speech_id,paragraph_len,document_id,speech_id,clean_speaker_name,prefix,document_title,hearing_date,...,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,ind
10608,7492,"He said, let's take an overall look at what w...",149789,114,1367,604,"Price, Tom",mr.,"military construction, veterans affairs, and r...","february 26, 2016",...,0.155191,0.078706,-0.000069,-0.024627,-0.207416,-0.084438,0.22280,-0.149085,-0.112272,train
10609,7492,"He said, let's take an overall look at what w...",149789,114,1367,604,"Price, Tom",mr.,"military construction, veterans affairs, and r...","february 26, 2016",...,0.155229,0.076456,0.023945,-0.036282,-0.167618,-0.094304,0.23413,-0.165397,-0.164193,train


---
### 3. Following sugestions

##### 1 - Merging the bills dataframe and the complete dataframe of scored paragraphs by the unique_speech_id column

In [3]:
print(f'''The unique_speech_id column in df_charisma_analysis has 
     {sum(df_charisma_analysis.unique_speech_id.value_counts()[df_charisma_analysis.unique_speech_id.value_counts()>1])} rows repeated''')
print(f'''     That is {sum(df_charisma_analysis.unique_speech_id.value_counts()[df_charisma_analysis.unique_speech_id.value_counts()>1])/df_charisma_analysis.shape[0]:.0%} of the rows repeated\n''')

print(f'''The unique_speech_id column in df_congress_114_bills has 
     {sum(df_congress_114_bills.unique_speech_id.value_counts()[df_congress_114_bills.unique_speech_id.value_counts()>1])} rows repeated''')

print(f'''     That is {sum(df_congress_114_bills.unique_speech_id.value_counts()[df_congress_114_bills.unique_speech_id.value_counts()>1])/df_congress_114_bills.shape[0]:.0%} of the rows repeated\n''')


The unique_speech_id column in df_charisma_analysis has 
     8353 rows repeated
     That is 76% of the rows repeated

The unique_speech_id column in df_congress_114_bills has 
     2278 rows repeated
     That is 59% of the rows repeated



If we merge the data using **unique_spech_id** the result will be:

In [4]:
df_merge_1=pd.merge(df_congress_114_bills, df_charisma_analysis,on='unique_speech_id')
print(f'''A data base with {df_merge_1.shape[0]} rows, where the unique_speech_id has {sum(df_merge_1.unique_speech_id.value_counts()[df_merge_1.unique_speech_id.value_counts()>1])} rows repeted''')

A data base with 1005 rows, where the unique_speech_id has 972 rows repeted


##### 2 - Merging the bills dataframe and the complete dataframe of scored paragraphs by the document_id and speech_id columns simultaneously 

In the case of **df_congress_114_bills** the index created by *document_id* and *speech_id*:

In [5]:
ind_1= df_congress_114_bills.document_id.astype(str)+'-'+df_congress_114_bills.speech_id.astype(str)
print(f'''produce {sum(ind_1.value_counts()[ind_1.value_counts()>1])} rows repeated of {df_congress_114_bills.shape[0]} ''')

produce 2278 rows repeated of 3886 


In the case of **df_charisma_analysis** the index created by *document_id* and *speech_id*:

In [6]:
ind_2= df_charisma_analysis.document_id.astype(str)+'-'+df_charisma_analysis.speech_id.astype(str)
print(f'''produce {sum(ind_2.value_counts()[ind_2.value_counts()>1])} rows repeated of {df_charisma_analysis.shape[0]} ''')

produce 8353 rows repeated of 10938 


When the two indexes cross:

In [7]:
ind_l1=set(ind_1.values)
ind_l2=set(ind_2.values)
print(f'''There are {sum([ a in ind_l2 for a in ind_l1])} elements of df_congres_114_bills index_adhoc in df_charisma_analysis''')

There are 123 elements of df_congres_114_bills index_adhoc in df_charisma_analysis


In [12]:
# generating the merging database:
df_congress_114_bills['index_adhoc']= df_congress_114_bills.document_id.astype(str)+'-'+df_congress_114_bills.speech_id.astype(str)
df_charisma_analysis['index_adhoc']= df_charisma_analysis.document_id.astype(str)+'-'+df_charisma_analysis.speech_id.astype(str)
df_merge_2=pd.merge(df_congress_114_bills,df_charisma_analysis, on='index_adhoc')
print(f'''A data base with {df_merge_2.shape[0]} rows, where the index_adhoc has {sum(df_merge_2.index_adhoc.value_counts()[df_merge_2.index_adhoc.value_counts()>1])} rows repeted''')

A data base with 1005 rows, where the index_adhoc has 972 rows repeted


##### 3 - Try looking only at the document_id and check that the same document_id exists in both the paragraphs and the sentences dataframes. 

In [101]:
doc_id_congress=[doc for doc in df_congress_114_bills.document_id.values] # creating list of doc_id from db_congress
doc_id_db_congress_in_db_charisma=[df_charisma_analysis[df_charisma_analysis.document_id==ele].shape[0] for ele in doc_id_congress]
doc_id_db_congress_in_db_charisma=pd.Series(data=doc_id_db_congress_in_db_charisma)
doc_id_db_congress_in_db_charisma.value_counts().sum

print(f'''There are  
      {len(doc_id_db_congress_in_db_charisma)-doc_id_db_congress_in_db_charisma.value_counts()[0]}
      document_id from db_congress_114 in db_charisma
      of which {doc_id_db_congress_in_db_charisma.value_counts()[1]} are unique''')

There are  
      1970
      document_id from db_congress_114 in db_charisma
      of which 273 are unique


In [100]:
doc_id_charisma=[doc for doc in df_charisma_analysis.document_id.values] # creating list of doc_id from db_congress
doc_id_db_charisma_in_db_congress=[df_congress_114_bills[df_congress_114_bills.document_id==ele].shape[0] for ele in doc_id_charisma]
doc_id_db_charisma_in_db_congress=pd.Series(data=doc_id_db_charisma_in_db_congress)
doc_id_db_charisma_in_db_congress.value_counts().sum

print(f'''There are  
      {len(doc_id_db_charisma_in_db_congress)-doc_id_db_charisma_in_db_congress.value_counts()[0]} 
      document_id from db_charisma in db_congress_114
      of which {doc_id_db_charisma_in_db_congress.value_counts()[1]} are unique''')

There are  
      2976 
      document_id from db_charisma in db_congress_114
      of which 956 are unique


> I don't see too much sense to continue with this suggestion because of the quantities of not unique in both db will mix the information

In [103]:
df_merge_3=pd.merge(df_charisma_analysis,df_congress_114_bills,on='document_id')
df_merge_3.shape

(23053, 199)

#### 4. New key 

> attribute sentences to the corresponding paragraphs. Both datasets refer to a series of documents which are identified by the column "name" which is the same in both datasets.

In [107]:
df_charisma_analysis.name.value_counts()

41.txt     293
1.txt      259
21.txt     240
47.txt     215
39.txt     212
          ... 
119.txt      2
107.txt      2
187.txt      1
110.txt      1
99.txt       1
Name: name, Length: 195, dtype: int64

In [109]:
df_congress_114_bills.name.value_counts()

24.txt     123
41.txt     112
4.txt      110
7.txt      102
51.txt      98
          ... 
183.txt      1
110.txt      1
125.txt      1
164.txt      1
176.txt      1
Name: name, Length: 167, dtype: int64

In [112]:
df_merge_4=pd.merge(df_congress_114_bills,df_charisma_analysis, on='name')
df_merge_4.name.value_counts()

41.txt     32816
24.txt     24723
4.txt      19690
39.txt     18868
1.txt      17353
           ...  
140.txt        6
181.txt        5
145.txt        3
192.txt        2
110.txt        1
Name: name, Length: 151, dtype: int64

> There are more *name* 195 in *charisma_analysis* than in *congress_114_bills* 167. Moreover, in common they only have 151!

So, there are **16** *name* in *congress_114_bills* not in *charisma_analysis*, and **44** *name* in *charisma_analysis* not in *congress_114_bills*.